In [ ]:
version 9 data update on 2 Feb 2024

In [ ]:
#population with 10 years dementia diagnosis
Create table `yhcr-prd-phm-bia-core.CB_MYSPACE_LG.0202_total` as
    SELECT distinct person_id,gender_source_value, birth_datetime, death_datetime,ethnicity_concept_id,  
    DATETIME_DIFF( '2009-01-01', birth_datetime, DAY)/365 as age
    FROM `yhcr-prd-phm-bia-core.CB_FDM_PrimaryCare_V9.person` 
WHERE DATETIME_DIFF( '2009-01-01', birth_datetime, DAY)/365 >=45
















DELETE FROM `yhcr-prd-phm-bia-core.CB_MYSPACE_LG.0202_total` WHERE death_datetime< '2009-01-01'













CREATE TABLE `yhcr-prd-phm-bia-core.CB_MYSPACE_LG.0202_total_impute`
as SELECT a.*, b.recode_ethnicity_concept_id
FROM `yhcr-prd-phm-bia-core.CB_MYSPACE_LG.0202_total` a Left Join  `yhcr-prd-phm-bia-core.CB_MYSPACE_LG.impute_ethnicity_1220` b
ON a.person_id=b.person_id

UPDATE `yhcr-prd-phm-bia-core.CB_MYSPACE_LG.0202_total_impute`
SET ethnicity_concept_id = recode_ethnicity_concept_id
WHERE ethnicity_concept_id = 8552

CREATE TABLE `yhcr-prd-phm-bia-core.CB_MYSPACE_LG.0202_total_impute_clean` as 
SELECT a.*
, b.gender_variable_value
, b.gender_variable_label
, c.ethnicity_variable_value_8levels
, c.ethnicity_variable_label_8levels
, d.dementia_variable_value
, d.dementia_variable_start_time
FROM `yhcr-prd-phm-bia-core.CB_MYSPACE_LG.0202_total_impute` as a 
LEFT join `yhcr-prd-phm-bia-core.CB_MYSPACE_LG.clean_gender` as b on a.gender_source_value = b.gender_source_value 
LEFT join `yhcr-prd-phm-bia-core.CB_MYSPACE_LG.clean_ethnic_1220` as c on a.ethnicity_concept_id = c.concept_id
left join  `yhcr-prd-phm-bia-core.CB_MYSPACE_LG.0202_dementia_10yrs` as d on a.person_id = d.person_id

#update dementia
Create table `yhcr-prd-phm-bia-core.CB_MYSPACE_LG.0218_total_dementia` as
Select distinct a.*
  , b.dementia_variable_start_time
  , b.dementia_variable_value
  from `yhcr-prd-phm-bia-core.CB_MYSPACE_LG.02_total_10yrs_over45` a
LEFT JOIN `yhcr-prd-phm-bia-core.CB_MYSPACE_LG.0218_dementia_10yrs` b
on a.person_id  = b.person_id

CREATE TABLE `yhcr-prd-phm-bia-core.CB_MYSPACE_LG.0202_total_impute_clean_age_index` as
Select distinct a.*
  , c.Lsoa
  , max(Index_of_Multiple_Deprivation_Decile)
  from `yhcr-prd-phm-bia-core.CB_MYSPACE_LG.0202_total_impute_clean_death_age` a
left join `yhcr-prd-phm-bia-core.CB_LOOKUPS.tbl_person_lsoa` as c 
on a.person_id = c.person_id
left join `yhcr-prd-phm-bia-core.CB_LOOKUPS.tbl_IMD_by_LSOA` as d 
on c.lsoa = d.LSOA_code

CREATE TABLE `yhcr-prd-phm-bia-core.CB_MYSPACE_LG.pj1_ten_year_completed` as
Select person_id, birth_datetime, death_datetime, ethnicity_concept_id, age, recode_ethnicity_concept_id, 
gender_variable_value, gender_variable_label, ethnicity_variable_value_8levels, ethnicity_variable_label_8levels, 
dementia_variable_value, dementia_variable_start_time, max(imd_max) max_index
FROM `yhcr-prd-phm-bia-core.CB_MYSPACE_LG.pj1_ten_year_impute_index`
GROUP BY person_id, birth_datetime, death_datetime, ethnicity_concept_id, age, recode_ethnicity_concept_id, 
gender_variable_value, gender_variable_label, ethnicity_variable_value_8levels, ethnicity_variable_label_8levels, 
dementia_variable_value, dementia_variable_start_time

In [1]:
rm(list=ls())
library(bigrquery)
library(tidyverse)
bq_auth()

project <- "yhcr-prd-phm-bia-core"
project_id <- "yhcr-prd-phm-bia-core"
source_dataset_pri <-'CB_FDM_PrimaryCare_V9'
source_dataset_pri <-gsub(' ','',source_dataset_pri)

print (source_dataset_pri)
target_dataset <- "CB_MYSPACE_LG"

sql_query <- "SELECT * FROM  `yhcr-prd-phm-bia-core.CB_MYSPACE_LG.0218_total_dementia`"
my_query <- bq_project_query(project_id, sql_query)
table <- bq_table_download(my_query)
write.csv(table, '0218_total.csv')

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.3     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


[1] "CB_FDM_PrimaryCare_V9"


In [2]:
rm(list=ls())
library(tidyverse)
library(lubridate)
library(dplyr) 
library(rio)
library(lubridate)
library(survival)
library(Hmisc)

data=import('total_10yrs_index_ages.csv')
print(nrow(data))


Attaching package: ‘Hmisc’


The following objects are masked from ‘package:dplyr’:

    src, summarize


The following objects are masked from ‘package:base’:

    format.pval, units




[1] 289218


In [3]:
data$birth_datetime<-as.Date(data$birth_datetime,format = "%m/%d/%Y")
data$death_datetime<-as.Date(data$death_datetime,format = "%m/%d/%Y")
data$time_start='2009-01-01'
data$time_start=as.Date(data$time_start)
table(data$death_datetime)
table(data$time_start)

data$dementia_variable_start_time=as.Date(data$dementia_variable_start_time,format = "%m/%d/%Y")
data$time_to_dementia<-(as.numeric(data$dementia_variable_start_time)-as.numeric(data$time_start))/365
data$time_to_dementia<-ifelse(is.na(data$time_to_dementia),10,data$time_to_dementia)
summary(data$time_to_dementia)

df=subset(data,data$time_to_dementia>0&data$time_to_dementia<=10) 
df$dementia=ifelse(is.na(df$dementia_variable_start_time),0,1)
table(df$dementia)
fit<-pyears(Surv(time = df$time_to_dementia,event = df$dementia)~ethnicity_variable_label_8levels,data=df, scale=1) 
fit
e<-fit$event
e
t<-fit$pyears
t
dt1=binconf(e,t,alpha = 0.05,method = "asymptotic")
dt1


2009-01-15 2009-02-15 2009-03-15 2009-04-15 2009-05-15 2009-06-15 2009-07-15 
       494        403        421        401        408        411        364 
2009-08-15 2009-09-15 2009-10-15 2009-11-15 2009-12-15 2010-01-15 2010-02-15 
       402        377        447        420        450        513        385 
2010-03-15 2010-04-15 2010-05-15 2010-06-15 2010-07-15 2010-08-15 2010-09-15 
       457        407        402        386        358        413        388 
2010-10-15 2010-11-15 2010-12-15 2011-01-15 2011-02-15 2011-03-15 2011-04-15 
       423        450        533        514        404        463        399 
2011-05-15 2011-06-15 2011-07-15 2011-08-15 2011-09-15 2011-10-15 2011-11-15 
       416        381        420        356        415        415        401 
2011-12-15 2012-01-15 2012-02-15 2012-03-15 2012-04-15 2012-05-15 2012-06-15 
       505        522        504        439        447        431        402 
2012-07-15 2012-08-15 2012-09-15 2012-10-15 2012-11-15 2012-12-


2009-01-01 
    289218 

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  0.000  10.000  10.000   9.061  10.000  11.003 


     0      1 
230017  58017 

Call:
pyears(formula = Surv(time = df$time_to_dementia, event = df$dementia) ~ 
    ethnicity_variable_label_8levels, data = df, scale = 1)

Total number of events: 49791 
Total number of person-years tabulated: 1766919 
Total number of person-years off table: 0 
Observations in the data set: 199818 
  (88216 observations deleted due to missingness)


ethnicity_variable_label_8levels
Asian British: East Asian and Others           Asian British: South Asian 
                                 337                                 7335 
              Black British: African  Black British: Caribbean and Others 
                                  96                                  334 
        Mixed/Multiple ethnic groups                   Other ethnic group 
                                 512                                  198 
                               White    White: Any other White background 
                               39563                                 1416

ethnicity_variable_label_8levels
Asian British: East Asian and Others           Asian British: South Asian 
                           18117.375                           195246.216 
              Black British: African  Black British: Caribbean and Others 
                            8750.351                            11516.701 
        Mixed/Multiple ethnic groups                   Other ethnic group 
                           23753.742                            16397.995 
                               White    White: Any other White background 
                         1419834.696                            73302.055

,PointEst,Lower,Upper
,0.01860093,0.016633538,0.02056832
,0.03756795,0.036724517,0.03841138
,0.01097099,0.008788446,0.01315353
,0.02900136,0.025936557,0.03206616
,0.02155450,0.019707700,0.02340130
,0.01207465,0.010402972,0.01374632
,0.02786451,0.027593793,0.02813523
,0.01931733,0.018320944,0.02031372


In [4]:
df=subset(data,data$time_to_dementia>0&data$time_to_dementia<=10) 
df$dementia=ifelse(is.na(df$dementia_variable_start_time),0,1)
table(df$dementia)
fit<-pyears(Surv(time = df$time_to_dementia,event = df$dementia)~ethnicity_variable_label_8levels
            ,data=df,subset=df$agegroup==4, scale=1) 
fit
e<-fit$event
e
t<-fit$pyears
t
dt1=binconf(e,t,alpha = 0.05,method = "asymptotic")
dt1


     0      1 
230017  58017 

Call:
pyears(formula = Surv(time = df$time_to_dementia, event = df$dementia) ~ 
    ethnicity_variable_label_8levels, data = df, subset = df$agegroup == 
    4, scale = 1)

Total number of events: 13864 
Total number of person-years tabulated: 253764.9 
Total number of person-years off table: 0 
Observations in the data set: 32583 
  (18482 observations deleted due to missingness)


ethnicity_variable_label_8levels
Asian British: East Asian and Others           Asian British: South Asian 
                                  44                                 1050 
              Black British: African  Black British: Caribbean and Others 
                                  13                                   89 
        Mixed/Multiple ethnic groups                   Other ethnic group 
                                 102                                   48 
                               White    White: Any other White background 
                               12000                                  518

ethnicity_variable_label_8levels
Asian British: East Asian and Others           Asian British: South Asian 
                           1122.2247                           18101.3452 
              Black British: African  Black British: Caribbean and Others 
                            273.9397                            1043.2356 
        Mixed/Multiple ethnic groups                   Other ethnic group 
                           1823.2904                            1273.3068 
                               White    White: Any other White background 
                         219219.3342                           10908.2493

,PointEst,Lower,Upper
,0.03920784,0.02785225,0.05056342
,0.05800674,0.05460144,0.06141204
,0.04745569,0.02227850,0.07263289
,0.08531150,0.06836043,0.10226258
,0.05594282,0.04539430,0.06649133
,0.03769712,0.02723568,0.04815856
,0.05473970,0.05378748,0.05569191
,0.04748700,0.04349589,0.05147810


In [3]:
df=subset(data,data$time_to_dementia>0&data$time_to_dementia<=10) 
df$dementia=ifelse(is.na(df$dementia_variable_start_time),0,1)
table(df$dementia)
fit<-pyears(Surv(time = df$time_to_dementia,event = df$dementia)~ethnicity_variable_label_8levels
            ,data=df, scale=1) 
fit
e<-fit$event
e
t<-fit$pyears
t
dt1=binconf(e,t,alpha = 0.05,method = "asymptotic")
dt1


     0      1 
230017  58017 

Call:
pyears(formula = Surv(time = df$time_to_dementia, event = df$dementia) ~ 
    ethnicity_variable_label_8levels, data = df, subset = df$agegroup == 
    4, scale = 1)

Total number of events: 13864 
Total number of person-years tabulated: 253764.9 
Total number of person-years off table: 0 
Observations in the data set: 32583 
  (18482 observations deleted due to missingness)


ethnicity_variable_label_8levels
Asian British: East Asian and Others           Asian British: South Asian 
                                  44                                 1050 
              Black British: African  Black British: Caribbean and Others 
                                  13                                   89 
        Mixed/Multiple ethnic groups                   Other ethnic group 
                                 102                                   48 
                               White    White: Any other White background 
                               12000                                  518

ethnicity_variable_label_8levels
Asian British: East Asian and Others           Asian British: South Asian 
                           1122.2247                           18101.3452 
              Black British: African  Black British: Caribbean and Others 
                            273.9397                            1043.2356 
        Mixed/Multiple ethnic groups                   Other ethnic group 
                           1823.2904                            1273.3068 
                               White    White: Any other White background 
                         219219.3342                           10908.2493

,PointEst,Lower,Upper
,0.03920784,0.02785225,0.05056342
,0.05800674,0.05460144,0.06141204
,0.04745569,0.02227850,0.07263289
,0.08531150,0.06836043,0.10226258
,0.05594282,0.04539430,0.06649133
,0.03769712,0.02723568,0.04815856
,0.05473970,0.05378748,0.05569191
,0.04748700,0.04349589,0.05147810


In [5]:
a1=96
b1=39563
a2=8750
b2=1419834
c1=a1/a2*1000
c2=b1/b2*1000

RR=c1/c2
d=log(RR)
lower=exp(d-1.96*sqrt((1/a1)+(1/a2)))
upper=exp(d+1.96*sqrt((1/a1)+(1/a2)))

RR
lower
upper

[1] 0.3937418

[1] 0.3220025

[1] 0.481464

In [ ]:
IRR <- rateratio(136, 1709, 22050, 127650)
 str(res)
 print(res)

In [ ]:
create table `yhcr-prd-phm-bia-core.CB_MYSPACE_LG.0202_total_impute_clean_age` as
select *,case when 45<= age AND age<= 55 then '1' 
when 55<= age AND age<= 65 then '2' 
when 65<= age AND age <= 75 then '3' 
else '4' end as agegroup
from `yhcr-prd-phm-bia-core.CB_MYSPACE_LG.0202_total_impute_clean`

In [ ]:
CREATE TABLE `yhcr-prd-phm-bia-core.CB_MYSPACE_LG.0202_total_impute_clean_age_index` as
Select distinct a.*
  , c.Lsoa
  , max(Index_of_Multiple_Deprivation_Decile)
  from `yhcr-prd-phm-bia-core.CB_MYSPACE_LG.0202_total_impute_clean_death_age` a
left join `yhcr-prd-phm-bia-core.CB_LOOKUPS.tbl_person_lsoa` as c 
on a.person_id = c.person_id
left join `yhcr-prd-phm-bia-core.CB_LOOKUPS.tbl_IMD_by_LSOA` as d 
on c.lsoa = d.LSOA_code

CREATE TABLE `yhcr-prd-phm-bia-core.CB_MYSPACE_LG.0202_total_impute_clean_age_index_slim` as
Select person_id, birth_datetime, death_datetime, ethnicity_concept_id, age, recode_ethnicity_concept_id, gender_variable_value,
gender_variable_label, ethnicity_variable_value_8levels, ethnicity_variable_label_8levels, dementia_variable_value,
dementia_variable_start_time, agegroup, age_at_death, max(Index_of_Multiple_Deprivation_Decile) max_index
FROM `yhcr-prd-phm-bia-core.CB_MYSPACE_LG.0202_total_impute_clean_age_index`
GROUP BY person_id, birth_datetime, death_datetime, ethnicity_concept_id, age, recode_ethnicity_concept_id,
gender_variable_value, gender_variable_label, ethnicity_variable_value_8levels, ethnicity_variable_label_8levels,
dementia_variable_value, dementia_variable_start_time, agegroup, age_at_death

In [72]:
library(dplyr)
df=subset(data) 
aggregate(df$age, list(df$ethnicity_variable_label_8levels), FUN=sd)

Group.1,x
<chr>,<dbl>
Asian British: East Asian and Others,9.978504
Asian British: South Asian,10.535255
Black British: African,8.586893
Black British: Caribbean and Others,11.402566
Mixed/Multiple ethnic groups,10.802404
Other ethnic group,10.725159
White,11.991537
White: Any other White background,12.720981


In [66]:

df %>%
  group_by(ethnicity_variable_label_8levels) %>%
  summarise_at(vars(age), list(name = mean),
              sd = sd(age))

ERROR: Error in is.data.frame(x): object 'age' not found


In [70]:
df %>%
  group_by(ethnicity_variable_label_8levels) %>%
  summarise(age = mean(age), sd = sd(df$age, na.rm=TRUE))

age,sd
<dbl>,<dbl>
62.1039,12.40171


In [76]:

df=subset(data,data$ethnicity_variable_label_8levels=NA) 
df %>%
  summarise(age = mean(age), sd = sd(df$age, na.rm=TRUE))

ERROR: Error in parse(text = x, srcfile = src): <text>:1:53: unexpected '='
1: df=subset(data,data$ethnicity_variable_label_8levels=
                                                        ^


In [ ]:
#female numbers

SELECT ethnicity_variable_label_8levels, count (distinct person_id) FROM `yhcr-prd-phm-bia-core.CB_MYSPACE_LG.0202_total_impute_clean_age_index_slim`  
WHERE gender_variable_value = 1
GROUP BY ethnicity_variable_label_8levels

#deprivation numbers 
SELECT ethnicity_variable_label_8levels, count (distinct person_id) FROM `yhcr-prd-phm-bia-core.CB_MYSPACE_LG.0202_total_impute_clean_age_index_slim`  
WHERE max_index = 1 OR max_index = 2
GROUP BY ethnicity_variable_label_8levels


In [ ]:
model2 <- lm(salary ~ yrs.service + rank + discipline + sex,
             data = Salaries)
Anova(model2)

In [26]:
df1<-subset(data,data$dementia_variable_value=1) 

ERROR: Error in parse(text = x, srcfile = src): <text>:1:46: unexpected '='
1: df1<-subset(data,data$dementia_variable_value=
                                                 ^


In [48]:
library(dplyr)
df=subset(data) 
model <- lm(age_onset ~ factor(ethnicity_variable_value_8levels)+age+gender_variable_value+max_index, data = df)
summary(model)$coef

,Estimate,Std. Error,t value,Pr(>|t|)
(Intercept),8.19920405,0.055675753,147.2670525,0.000000e+00
factor(ethnicity_variable_value_8levels)2,0.02443234,0.045157121,0.5410519,5.884746e-01
factor(ethnicity_variable_value_8levels)3,-0.02625255,0.022439377,-1.1699322,2.420346e-01
factor(ethnicity_variable_value_8levels)4,-0.12658971,0.090665664,-1.3962255,1.626536e-01
factor(ethnicity_variable_value_8levels)5,0.25918002,0.163815345,1.5821474,1.136231e-01
factor(ethnicity_variable_value_8levels)6,-0.06795913,0.089903519,-0.7559118,4.497061e-01
factor(ethnicity_variable_value_8levels)7,-0.02455543,0.072423745,-0.3390522,7.345720e-01
factor(ethnicity_variable_value_8levels)8,0.04715161,0.117897058,0.3999388,6.892035e-01
age,0.95685949,0.000793894,1205.2735964,0.000000e+00
gender_variable_value,0.09573753,0.015244573,6.2801058,3.414923e-10
